# US Presidential Election Analysis: Electoral College, Popular Vote, or Both?

## Objective
This notebook contains the first step in a larger effort to analyze historical US presidential election data. It focuses on scraping the electoral college voting results for all modern US presidential elections (i.e. from 1892 to the present) from the [National Archives Website](https://www.archives.gov/electoral-college/results), and then writing the results to a Postgres Database for subsequent analysis. The following steps are implemented in this notebook:
1. [X] Initial Setup: Import Modules, Define Functions, and Set Parameter Values
2. [X] Scrape Electoral College Data from the National Archives Website
    1. Define Set containing All Presidential Election Years
    2. Define Set containing All US "States" that Vote in Presidential Elections (includes Washington DC)
    3. Scrape National Archive Summary web page for Links to each Election Year's Data
    4. Scrape each Election Year's web page to download the two tables containing all Election Data
    5. Parse the Data for All Election Years into a useable, compact format
    6. Validate Accuracy of Parsed Election Data
3. [ ] Transform and Validate Parsed Election Data so that it conforms to a Star Schema design pattern, with Candidate and State Dimension Tables and Electoral Vote Fact table:
    1. Spot Check Parsed Data for Individual Election Years
    2. Transform and Validate Candidate Data
    3. Transform and Validate State Data
    4. Transform and Validate Electoral Votes Data
4. [ ] Write Election Data Tables to Postgres

### Notes
- The National Archives website only contains Electoral College results for US Presidential Elections from 1892 to present. Another data source will be scraped to get Popular Vote data for each Presidential Election
- Currently I'm only scraping information regarding the Presidential Candidates and their electoral college vote tallies; however, Vice Presidential results are also available, so I can circle back to include that data if the need arises
- FYI: Any candidate who wins a majority or plurality of the popular vote nationwide has a good chance of winning in the Electoral College, but there are no guarantees: for example the results of 1824, 1876, 1888, 2000, and 2016 elections (see Reference 1 below).

### Data Sources
1. US Presidential Election Electoral College voting results: https://www.archives.gov/electoral-college/results
2. US States shapefile: https://www2.census.gov/geo/tiger/TIGER2019/STATE/tl_2019_us_state.zip

### References
1. Electoral College History: https://www.archives.gov/electoral-college/history
2. https://towardsdatascience.com/scraping-table-data-from-websites-using-a-single-line-in-python-ba898d54e2bc
3. https://searchdatamanagement.techtarget.com/definition/star-schema

## 1. Setup

### 1.1 Import Modules

In [1]:
# import modules
from bs4 import BeautifulSoup
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests

### 1.2 Define Functions

In [369]:
def get_html_tables(url, div_id="main-col", find_all=False):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    div = soup.find("div", id=div_id)
    if find_all:
        return div.find_all("table")
    else:
        return div.find("table")
    
def scrape_election_links(archive_url_domain, archive_url_base):
    link_table = get_html_tables(archive_url_domain+archive_url_base)
    return [archive_url_domain+a['href'] for a in link_table.find_all("a")]    

def scrape_raw_election_tables(election_links, us_election_years):
    raw_election_tables = {}
    for link in election_links:
        link_year = int(link.split('/')[-1])
        if link_year in us_election_years:
            raw_election_tables[link_year] = get_html_tables(link, find_all=True)
        else:
            print(f"Error: The link year, {link_year}, parsed from the following link does not match a US election year: \n{link}")
    return raw_election_tables
    
def parse_election_years(data_tables, state_names):
    parsed_years = []
    for ind, year in enumerate(data_tables.keys()):
        print(f"Working on Election Year = {year} ({ind})")
        parsed_dict = parse_election_year_tables(data_tables[year], state_names)
        parsed_dict['year'] = year
        parsed_years.append(parsed_dict)
    return parsed_years

def parse_election_year_tables(year_tables, state_names):
    parsed_tables = {}
    parsed_tables['t1'] = parse_table1(year_tables[0].find_all('tr'))
    parsed_tables['t2'] = parse_table2(year_tables[1].find_all('tr'), state_names)
    return parsed_tables

def parse_table1(t1_rows):
    cp_row_inds = [0, 1]
    cp_row_headers = ["President", "Main Opponent"]
    candidate_party = []
    for ri, rh in zip(cp_row_inds, cp_row_headers):
        candidate_party.append(parse_t1_candidate_party(t1_rows, ri, rh))
    return candidate_party
    
# Parse Table 1 to store the Presidential Candidates Name and Party
def parse_t1_candidate_party(t1_rows, row_ind, row_header):
    if t1_rows[row_ind].find('th').get_text() == row_header:
        row_data = t1_rows[row_ind].find('td').get_text()
        candidate, party = row_data.split(' [')
        return { \
            'president_candidate_name': candidate.strip(" *").replace(",", ""), \
            'president_candidate_party': party.strip(" *]") \
        }
        #return (candidate.strip(" *"), party.strip(" *]"))
    else:
        print(f"Error: Row{row_ind} does not contain data for {row_header}")

def parse_table2(t2_rows, state_names):
    t2_data = {}
    num_candidates = parse_t2_num_candidates(t2_rows[0])
    t2_data['candidate_state'] = parse_t2_candidate_state(t2_rows[1], num_candidates)
    t2_data['votes_by_state'] = parse_t2_votes_by_state(t2_rows[2:], num_candidates, state_names)
    return t2_data

def parse_t2_num_candidates(header_row):
    return int(header_row.find('th', text="For President").get('colspan'))
    
def parse_t2_candidate_state(cs_row, num_candidates):
    cs_cols = cs_row.find_all('td')
    candidate_state = []
    for ci, cs in enumerate(cs_cols[:num_candidates]):
        if cs.find('br'):
            text = " ".join(cs.stripped_strings)
        else:
            text = cs.get_text()
        if text == "Other":
            candidate, state = text, None
        else:
            candidate, state = text.split(" of ")
            candidate = candidate.strip(", *").replace(",", "")
            state = state.strip(" *").replace(",", "")
        candidate_state.append({ \
            'president_candidate_name': candidate, \
            'col_ind': ci+1, \
            'president_candidate_state': state \
        })
        #candidate_state.append((candidate.strip(", *"), state.strip(", *")))
    return candidate_state

def parse_t2_votes_by_state(states_rows, num_candidates, state_names):
    votes_by_state = []
    for sr in states_rows:
        state_cols = sr.find_all('td')
        col_0_text = state_cols[0].get_text().strip(" *")
        if col_0_text in state_names:
            state = col_0_text
            start_ind, end_ind = 1, num_candidates+2
        elif col_0_text in {"Total", "Totals"}:
            state = "Totals"
            start_ind, end_ind = 1, num_candidates+2
        elif sr.find('th', text='Total'):
            state = "Totals"
            start_ind, end_ind = 0, num_candidates+1
        else:
            state = False
        # Only parse vote data and store it if a valid state value is found
        # This helps validate the state name was parsed correctly, and
        # skips the Notes row at the end of some of the tables
        if state:
            state_votes = {'state': state}
            for si, sv in enumerate(state_cols[start_ind:end_ind]):
                votes = sv.get_text()
                if si == 0:
                    si = 'total_electoral_votes'
                state_votes[si] = int(votes) if votes != '-' else 0
            votes_by_state.append(state_votes)
            # state_votes = [state]   
            # for si, sv in enumerate(state_cols[start_ind:end_ind]):
            #     votes = sv.get_text()
            #     state_votes.append(int(votes) if votes != '-' else 0)
            # votes_by_state.append(tuple(state_votes))
    return votes_by_state

def print_election_year_results(parsed_year):
    print(f"Election Year: {parsed_year['year']}")
    print(f"Table 1 Top 2 Candidates + Party: \n{pprint_list_of_dicts(parsed_year['t1'])}")
    print(f"Table 2 Candidates + Home State: \n{pprint_list_of_dicts(parsed_year['t2']['candidate_state'])}")
    print(f"Table 2 Votes by State: \n{pprint_list_of_dicts(parsed_year['t2']['votes_by_state'])}")
    
def pprint_list_of_dicts(list_of_dicts):
    list_of_strings = [str(d).strip('{}') for d in list_of_dicts]
    return '\t'+'\n\t'.join(list_of_strings)

def get_name_middle_last(middle_last):
    try:
        split = middle_last.split()
    except:
        split = []
    if len(split) == 1:
        return (None, split[0])
    elif len(split) > 1:
    # This assumes any space in the name is part of the last name
        return (split[0], " ".join(split[1:]))
    else:
        return (None, None)

def make_map_usa(df, col2plot, figsize=(15,8), title=None, fontsize=18, cmap='Blues', edgecolor='k'):
    # Define fixed longitude limit, xlim, and latitude limit, ylim, for USA
    xlim = (-172, -58)
    ylim = (16, 74)
    fig, ax = plt.subplots(1, figsize=figsize)
    df.plot(column=col2plot, ax=ax, cmap=cmap, edgecolor=edgecolor)
    ax.axis('off')
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=df[col2plot].min(), vmax=df[col2plot].max()))
    sm._A = []
    cb = fig.colorbar(sm)
    cb.set_label(col2plot, fontsize=fontsize)
    cb.ax.tick_params(labelsize=fontsize)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    if title:
        ax.set_title(title, fontsize=fontsize+4)
    plt.tight_layout()

### 1.3 Set Parameters

Define the base URL for the National Archives, and the resource location for the summary page containing links to Presidential election data for each year. The shape file containing US State data can be downloaded from [this link](https://www2.census.gov/geo/tiger/TIGER2019/STATE/).

In [3]:
latest_election_year = 2020
archive_url_domain = "https://www.archives.gov"
archive_url_base = "/electoral-college/results"
usa_state_shp = "/home/fdpearce/Documents/Projects/data/Maps/State_Shapes/tl_2019_us_state/tl_2019_us_state.shp"

## 2. Scrape Electoral College Data from the National Archives Website

### 2.1 Define Set Containing All Presidential Election Years

Create a set, us_election_years, with every year that a US Presidential Election occurred. This set will be used to scrape all available election data. See the `archive_url` website for the complete list of election years. A set is used so membership checks execute efficiently (O(1)).

In [127]:
us_election_years = [1789]+list(range(1792, latest_election_year+4, 4))
print(*us_election_years)

1789 1792 1796 1800 1804 1808 1812 1816 1820 1824 1828 1832 1836 1840 1844 1848 1852 1856 1860 1864 1868 1872 1876 1880 1884 1888 1892 1896 1900 1904 1908 1912 1916 1920 1924 1928 1932 1936 1940 1944 1948 1952 1956 1960 1964 1968 1972 1976 1980 1984 1988 1992 1996 2000 2004 2008 2012 2016 2020


In [5]:
us_election_years = set(us_election_years)

### 2.2 Define Set Containing All US States

Load all the data from the USA States shape file. For now, only the state names are extracted, but later the geometry column can be used to generate maps, extract state features, etc. The names of US Territories that don't participate in Presidential Elections are dropped from the variable containing the set of states.

In [6]:
usa = gpd.read_file(usa_state_shp)
us_state_names = usa['NAME'].values
print(sorted(us_state_names))

['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Commonwealth of the Northern Mariana Islands', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'United States Virgin Islands', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [7]:
us_state_names = set(us_state_names)
print(f"Total # of States = {len(us_state_names)}")

Total # of States = 56


In [8]:
territories = ['American Samoa', 'Commonwealth of the Northern Mariana Islands', 'Guam', 'Puerto Rico', 'United States Virgin Islands']
us_state_names.difference_update(territories)
print(f"Total # of States = {len(us_state_names)}")

Total # of States = 51


### 2.3 Scrape the Links to each Election Year's Data

Parse html summary page and extract links to each page containing data for a given Presidential Election Year.

In [9]:
election_links = scrape_election_links(archive_url_domain, archive_url_base)
print(election_links)

['https://www.archives.gov/electoral-college/1892', 'https://www.archives.gov/electoral-college/1896', 'https://www.archives.gov/electoral-college/1900', 'https://www.archives.gov/electoral-college/1904', 'https://www.archives.gov/electoral-college/1908', 'https://www.archives.gov/electoral-college/1912', 'https://www.archives.gov/electoral-college/1916', 'https://www.archives.gov/electoral-college/1920', 'https://www.archives.gov/electoral-college/1924', 'https://www.archives.gov/electoral-college/1928', 'https://www.archives.gov/electoral-college/1932', 'https://www.archives.gov/electoral-college/1936', 'https://www.archives.gov/electoral-college/1940', 'https://www.archives.gov/electoral-college/1944', 'https://www.archives.gov/electoral-college/1948', 'https://www.archives.gov/electoral-college/1952', 'https://www.archives.gov/electoral-college/1956', 'https://www.archives.gov/electoral-college/1960', 'https://www.archives.gov/electoral-college/1964', 'https://www.archives.gov/elec

### 2.4 Scrape the Two Tables Containing each Election Year's Data

Data tables dict has keys for each year data is available. Each value is a list with html for the two tables containing election data, which are stored in their own variables primarily for debugging purposes.

In [10]:
raw_election_tables = scrape_raw_election_tables(election_links, us_election_years)

In [126]:
print("Data is currently available from the National Archives website for the following election years:")
print(*raw_election_tables.keys(), sep=", ")

Data is available from the National Archives website for the following election years:
1892, 1896, 1900, 1904, 1908, 1912, 1916, 1920, 1924, 1928, 1932, 1936, 1940, 1944, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020


### 2.5 Parse the Data for All Election Years

In [370]:
parsed_election_years = parse_election_years(raw_election_tables, us_state_names)

Working on Election Year = 1892 (0)
Working on Election Year = 1896 (1)
Working on Election Year = 1900 (2)
Working on Election Year = 1904 (3)
Working on Election Year = 1908 (4)
Working on Election Year = 1912 (5)
Working on Election Year = 1916 (6)
Working on Election Year = 1920 (7)
Working on Election Year = 1924 (8)
Working on Election Year = 1928 (9)
Working on Election Year = 1932 (10)
Working on Election Year = 1936 (11)
Working on Election Year = 1940 (12)
Working on Election Year = 1944 (13)
Working on Election Year = 1948 (14)
Working on Election Year = 1952 (15)
Working on Election Year = 1956 (16)
Working on Election Year = 1960 (17)
Working on Election Year = 1964 (18)
Working on Election Year = 1968 (19)
Working on Election Year = 1972 (20)
Working on Election Year = 1976 (21)
Working on Election Year = 1980 (22)
Working on Election Year = 1984 (23)
Working on Election Year = 1988 (24)
Working on Election Year = 1992 (25)
Working on Election Year = 1996 (26)
Working on 

## 3. Transform and Validate Parsed Election Data
The data transformation performed in this section ultimately creates three tables, following a star schema design (see Reference 3 above):
1. Candidate dimension table built in Section 3.2
2. State dimension table built in Section 3.3
3. Votes by Year fact table built in Section 3.4

Data validation is performed in Section 3.1, and in each of the sections outlined above.

### 3.1 Spot Check Parsed Data for Individual Election Years
Print a compact view of the data parsed for a given election year. The year index value is available in the results of the previous cell. This output provides a useful reference of the parsed data structure for a given election year that is used for development of later sections.

In [558]:
year_index = 13
print_election_year_results(parsed_election_years[year_index])

Election Year: 1944
Table 1 Top 2 Candidates + Party: 
	'president_candidate_name': 'Franklin D. Roosevelt', 'president_candidate_party': 'D'
	'president_candidate_name': 'Thomas E. Dewey', 'president_candidate_party': 'R'
Table 2 Candidates + Home State: 
	'president_candidate_name': 'Franklin D. Roosevelt', 'col_ind': 1, 'president_candidate_state': 'New York'
	'president_candidate_name': 'Thomas E. Dewey', 'col_ind': 2, 'president_candidate_state': 'New York'
Table 2 Votes by State: 
	'state': 'Alabama', 'total_electoral_votes': 11, 1: 11, 2: 0
	'state': 'Arizona', 'total_electoral_votes': 4, 1: 4, 2: 0
	'state': 'Arkansas', 'total_electoral_votes': 9, 1: 9, 2: 0
	'state': 'California', 'total_electoral_votes': 25, 1: 25, 2: 0
	'state': 'Colorado', 'total_electoral_votes': 6, 1: 0, 2: 6
	'state': 'Connecticut', 'total_electoral_votes': 8, 1: 8, 2: 0
	'state': 'Delaware', 'total_electoral_votes': 3, 1: 3, 2: 0
	'state': 'Florida', 'total_electoral_votes': 8, 1: 8, 2: 0
	'state': 'Geo

### 3.2 Transform and Validate Candidate Data
This section creates a Candidate fact table that will be written to Postgres in subsequent steps. I start with the Candidate State data from Table 2 as that gives the complete list of presidential candidates that received electoral votes, except for the troubling 2016 election that has values of other, due to the unprecedented number of presidential candidates that received electoral votes. I'd need to circle back to parse the Notes section on Table 2 to get all candidate names that received at least one electoral vote in 2016, and it may simply be easier to enter that info manually at a later date...

#### **3.2.1 Extract, Validate and Transform Table 2 Data**

In [642]:
t2_states_df = pd.json_normalize(parsed_election_years, ['t2', 'candidate_state'], ['year'])

In [643]:
t2_states_df.head()

,president_candidate_name,col_ind,president_candidate_state,year
0,Grover Cleveland,1,New York,1892
1,Benjamin Harrison,2,Indiana,1892
2,James B. Weaver,3,Iowa,1892
3,William McKinley,1,Ohio,1896
4,William J. Bryan,2,Nebraska,1896


In [644]:
t2_states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   president_candidate_name   79 non-null     object
 1   col_ind                    79 non-null     int64 
 2   president_candidate_state  77 non-null     object
 3   year                       79 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.6+ KB


In [645]:
t2_states_df[t2_states_df['president_candidate_state'].isna()]

,president_candidate_name,col_ind,president_candidate_state,year
74,Other,2,None,2016
76,Other,4,None,2016


#### Build Initial Candidates DataFrame

In [785]:
candidates_df = t2_states_df[['president_candidate_name', 'president_candidate_state']].drop_duplicates().reset_index(drop=True)

In [786]:
num_can_st = len(t2_states_df[['president_candidate_name', 'president_candidate_state']])
print(f"Original number of Candidate-State combinations = {num_can_st}")
num_unique_can_st = len(candidates_df)
print(f"Unique number of Candidate-State combinations = {num_unique_can_st}")
num_unique_can = len(candidates_df.drop_duplicates(subset='president_candidate_name'))
print(f"Unique number of Candidates = {num_unique_can}")

Original number of Candidate-State combinations = 79
Unique number of Candidate-State combinations = 57
Unique number of Candidates = 56


#### Update Column Names

In [787]:
# Remove 'president_candidate_' prefix and change 'name' to 'full_name'
candidates_df.columns = candidates_df.columns.str.replace("president_candidate_", "")

#### Use Full Name to Create Columns for First, Middle, Last, and Suffix of Name

In [788]:
# Take first split on candidate name to get first name (that's the easy one, lol)
candidates_df[['name_first', 'name_remainder']] = candidates_df['name'].str.split(n=1, expand=True)

In [789]:
# Remove Jr. suffix, commas, leading/trailing spaces from name_remainder prior to parsing middle/last name
candidates_df['name_remainder'] = candidates_df['name_remainder'].str.replace(r",? Jr\.?$", "", regex=True)

In [790]:
# Use get_name_middle_last function to split name_remainder into middle and last name
candidates_df[['name_middle', 'name_last']] = pd.DataFrame(candidates_df['name_remainder'].map(get_name_middle_last).tolist(), index=candidates_df.index)
candidates_df.drop('name_remainder', axis=1, inplace=True)

In [791]:
# Finally add column containing suffix "Jr." if present in full name
candidates_df['name_suffix'] = candidates_df['name'].map(lambda x: "Jr." if x.endswith("Jr.") else None)
candidates_df

,name,state,name_first,name_middle,name_last,name_suffix
0,Grover Cleveland,New York,Grover,None,Cleveland,None
1,Benjamin Harrison,Indiana,Benjamin,None,Harrison,None
2,James B. Weaver,Iowa,James,B.,Weaver,None
3,William McKinley,Ohio,William,None,McKinley,None
4,William J. Bryan,Nebraska,William,J.,Bryan,None
5,Theodore Roosevelt,New York,Theodore,None,Roosevelt,None
6,Alton B. Parker,New York,Alton,B.,Parker,None
7,William H. Taft,Ohio,William,H.,Taft,None
8,Woodrow Wilson,New Jersey,Woodrow,None,Wilson,None
9,Charles E. Hughes,New York,Charles,E.,Hughes,None


#### Validate there are no duplicate candidate names
Duplicate candidate names within this context refers to a candidate that has two different full name spellings in two or more election years, but it is the same candidate. As the table above shows
- **The two Donald Trump rows are true duplicates within this context and need to be updated**
- The two George Bush rows are expected as these are the father-son pair
- The two Richard Nixon rows duplicates are due to his two state affiliations, which will be handled later in this section

In [792]:
name_first_last = candidates_df.groupby(['name_first', 'name_last'], as_index=False).size()
name_first_last[name_first_last['size']>1]

,name_first,name_last,size
10,Donald,Trump,2
13,George,Bush,2
38,Richard,Nixon,2


In [793]:
candidates_df[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')]

,name,state,name_first,name_middle,name_last,name_suffix
52,Donald Trump,New York,Donald,None,Trump,None
56,Donald J. Trump,Florida,Donald,J.,Trump,None


#### Correct Donald Trump's full name and middle initial on row for New York

In [794]:
candidates_df.loc[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')&(candidates_df['state']=='New York'), 'name'] = \
candidates_df.loc[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')&(candidates_df['state']=='Florida'), 'name'].values[0]
candidates_df.loc[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')&(candidates_df['state']=='New York'), 'name_middle'] = \
candidates_df.loc[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')&(candidates_df['state']=='Florida'), 'name_middle'].values[0]

In [795]:
candidates_df[(candidates_df['name_first']=='Donald')&(candidates_df['name_last']=='Trump')]

,name,state,name_first,name_middle,name_last,name_suffix
52,Donald J. Trump,New York,Donald,J.,Trump,None
56,Donald J. Trump,Florida,Donald,J.,Trump,None


#### Aggregate Candidates with Multiple State Affiliations
Group by candidate name to obtain a DataFrame at the correct grain, i.e. one row per candidate. For candidates with more than one state affiliation, combine their state affiliations into a single column, 'state', with each state separated by a hyphen. The current approach produces the desired result without the need for presorting the state values because I want the first election year's party affiliation to be the primary one.
- **Richard M. Nixon and Donald J. Trump are the only two candidate with more than one State association**

In [796]:
state_row_count = candidates_df.groupby('name', as_index=False).size()
state_row_count[state_row_count['size']>1]

,name,size
10,Donald J. Trump,2
40,Richard M. Nixon,2


In [797]:
candidates_df[(candidates_df['name']=="Richard M. Nixon")|(candidates_df['name']=="Donald J. Trump")]

,name,state,name_first,name_middle,name_last,name_suffix
27,Richard M. Nixon,California,Richard,M.,Nixon,None
31,Richard M. Nixon,New York,Richard,M.,Nixon,None
52,Donald J. Trump,New York,Donald,J.,Trump,None
56,Donald J. Trump,Florida,Donald,J.,Trump,None


In [798]:
candidates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         57 non-null     object
 1   state        56 non-null     object
 2   name_first   57 non-null     object
 3   name_middle  36 non-null     object
 4   name_last    56 non-null     object
 5   name_suffix  2 non-null      object
dtypes: object(6)
memory usage: 2.8+ KB


In [799]:
candidates_df = candidates_df.groupby(['name', 'name_first', 'name_middle', 'name_last', 'name_suffix'], sort=False, dropna=False)['state'].agg( \
                                                                                                state=lambda x: "-".join(i if i is not None else "" for i in x)).reset_index()
candidates_df

,name,name_first,name_middle,name_last,name_suffix,state
0,Grover Cleveland,Grover,NaN,Cleveland,NaN,New York
1,Benjamin Harrison,Benjamin,NaN,Harrison,NaN,Indiana
2,James B. Weaver,James,B.,Weaver,NaN,Iowa
3,William McKinley,William,NaN,McKinley,NaN,Ohio
4,William J. Bryan,William,J.,Bryan,NaN,Nebraska
5,Theodore Roosevelt,Theodore,NaN,Roosevelt,NaN,New York
6,Alton B. Parker,Alton,B.,Parker,NaN,New York
7,William H. Taft,William,H.,Taft,NaN,Ohio
8,Woodrow Wilson,Woodrow,NaN,Wilson,NaN,New Jersey
9,Charles E. Hughes,Charles,E.,Hughes,NaN,New York


#### Parse State Column to Create Primary, Secondary State Columns

In [800]:
# Take first split on candidate name to get first name (that's the easy one, lol)
candidates_df[['state', 'state_2']] = candidates_df['state'].str.split("-", n=1, expand=True)
candidates_df.loc[candidates_df['state']=="", 'state'] = None
candidates_df

,name,name_first,name_middle,name_last,name_suffix,state,state_2
0,Grover Cleveland,Grover,NaN,Cleveland,NaN,New York,None
1,Benjamin Harrison,Benjamin,NaN,Harrison,NaN,Indiana,None
2,James B. Weaver,James,B.,Weaver,NaN,Iowa,None
3,William McKinley,William,NaN,McKinley,NaN,Ohio,None
4,William J. Bryan,William,J.,Bryan,NaN,Nebraska,None
5,Theodore Roosevelt,Theodore,NaN,Roosevelt,NaN,New York,None
6,Alton B. Parker,Alton,B.,Parker,NaN,New York,None
7,William H. Taft,William,H.,Taft,NaN,Ohio,None
8,Woodrow Wilson,Woodrow,NaN,Wilson,NaN,New Jersey,None
9,Charles E. Hughes,Charles,E.,Hughes,NaN,New York,None


#### **Validate Grain is Correct: One Candidate per Row**

In [801]:
print(f"Q: Does each row correspond to a unique candidate name? A: {len(candidates_df) == len(candidates_df.drop_duplicates(subset='name'))}")

Q: Does each row correspond to a unique candidate name? A: True


#### **Validate State Distributions are Correct**

In [802]:
num_candidates = len(candidates_df)

In [804]:
state_counts = candidates_df['state'].value_counts(dropna=False)
state_counts

New York          11
Massachusetts      5
California         4
Ohio               4
Texas              4
Minnesota          2
Arizona            2
Alabama            2
Kansas             2
Illinois           2
Wisconsin          1
South Dakota       1
NaN                1
North Carolina     1
Tennessee          1
Arkansas           1
Michigan           1
Georgia            1
Iowa               1
Missouri           1
Nebraska           1
New Jersey         1
Virginia           1
West Virginia      1
Indiana            1
South Carolina     1
Delaware           1
Name: state, dtype: int64

In [805]:
state_2_counts = candidates_df['state_2'].value_counts(dropna=False)
state_2_counts

NaN         53
New York     1
Florida      1
Name: state_2, dtype: int64

#### **3.2.2 Extract, Validate and Transform Table 1 Candidate Data**

In [816]:
t1_df = pd.json_normalize(parsed_election_years, 't1', ['year'])

In [817]:
t1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   president_candidate_name   66 non-null     object
 1   president_candidate_party  66 non-null     object
 2   year                       66 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB


In [818]:
t1_df.head()

,president_candidate_name,president_candidate_party,year
0,Grover Cleveland,D,1892
1,Benjamin Harrison,R,1892
2,William McKinley,R,1896
3,William J. Bryan,D-P,1896
4,William McKinley,R,1900


In [819]:
t1_df['president_candidate_party'].value_counts()

R      32
D      31
D-P     2
P       1
Name: president_candidate_party, dtype: int64

In [820]:
num_can_pa = len(t1_df[['president_candidate_name', 'president_candidate_party']])
print(f"Original number of Candidate-Party combinations = {num_can_pa}")
num_unique_can_pa = len(t1_df.drop_duplicates(subset=['president_candidate_name', 'president_candidate_party']))
print(f"Unique number of Candidate-Party combinations = {num_unique_can_pa}")
num_unique_can = len(t1_df.drop_duplicates(subset='president_candidate_name'))
print(f"Unique number of Candidates = {num_unique_can}")

Original number of Candidate-Party combinations = 66
Unique number of Candidate-Party combinations = 47
Unique number of Candidates = 45


In [821]:
# William J. Bryan has two party affiliations: Primary = "D" and Secondary = "P"
t1_df[t1_df['president_candidate_name']=="William J. Bryan"]

,president_candidate_name,president_candidate_party,year
3,William J. Bryan,D-P,1896
5,William J. Bryan,D-P,1900
9,William J. Bryan,D,1908


In [822]:
# Only William J. Bryan has a split party designation
t1_df[t1_df['president_candidate_party'].str.contains("-")]

,president_candidate_name,president_candidate_party,year
3,William J. Bryan,D-P,1896
5,William J. Bryan,D-P,1900


In [823]:
# Theodore Roosevelt is the only candidate to change parties: Primary = "R" and Secondary = "P"
t1_df[t1_df['president_candidate_name']=='Theodore Roosevelt']

,president_candidate_name,president_candidate_party,year
6,Theodore Roosevelt,R,1904
11,Theodore Roosevelt,P,1912


#### Build Candidates Party DataFrame
This Candidate table will be joined to the one above to add party affiliation if available.

In [824]:
candidates_party_df = t1_df[['president_candidate_name', 'president_candidate_party']].drop_duplicates().reset_index(drop=True)

#### Update Column Names

In [825]:
# Remove 'president_candidate_' prefix and change 'name' to 'full_name'
candidates_party_df.columns = candidates_party_df.columns.str.replace("president_candidate_", "")
candidates_party_df.head()

,name,party
0,Grover Cleveland,D
1,Benjamin Harrison,R
2,William McKinley,R
3,William J. Bryan,D-P
4,Theodore Roosevelt,R


#### Aggregate Candidates with Multiple Party Affiliations
Combine all party affiliations into a single column, 'party', separated by hyphens. This aggregation also yields a DataFrame at the correct grain, i.e. one row per candidate. Note that this produces the desired result without the need for sorting as I want the first party affiliation to be the primary one. May need to revisit this assumption in the future...

In [826]:
candidates_party_df = candidates_party_df.groupby('name')['party'].agg(party="-".join).reset_index()

#### Parse Party Column to Create Primary, Secondary Party Columns

In [827]:
# Take first split on candidate name to get first name (that's the easy one, lol)
candidates_party_df[['party', 'party_2']] = candidates_party_df['party'].str.split("-", n=1, expand=True)
candidates_party_df['party_2'] = candidates_party_df['party_2'].map(lambda x: x[0] if x else None)
candidates_party_df

,name,party,party_2
0,Adlai Stevenson,D,None
1,Albert Gore Jr.,D,None
2,Alfred E. Smith,D,None
3,Alfred M. Landon,R,None
4,Alton B. Parker,D,None
5,Barack Obama,D,None
6,Barry M. Goldwater,R,None
7,Benjamin Harrison,R,None
8,Bob Dole,R,None
9,Calvin Coolidge,R,None


#### Validate Grain is Correct: One Candidate per Row

In [828]:
print(f"Q: Does each row correspond to a unique candidate name? A: {len(candidates_party_df) == len(candidates_party_df.drop_duplicates(subset='name'))}")

Q: Does each row correspond to a unique candidate name? A: True


#### Validate Party Distributions are Correct

In [830]:
party_counts = candidates_party_df['party'].value_counts(dropna=True)
party_counts

D    23
R    22
Name: party, dtype: int64

In [831]:
party_2_counts = candidates_party_df['party_2'].value_counts(dropna=True)
party_2_counts

P    2
Name: party_2, dtype: int64

#### **3.2.3 Check Consistency of Names from Table 1 and Table 2**

Full names that don't match for the same candidate between the two datasets need to be fixed so that the joining of the two datasets works correctly.
- Update 'Bob Dole in Table 1 to 'Robert Dole' so it matches the name value in Table 2
- Update 'George McGovern' in Table 2 to 'George S. McGovern' so it matches the name value in Table 1, and update his 'name_middle' value to 'S. in Table 2

In [832]:
t1_names = set(candidates_party_df['name'].unique())
t1_t2_name_diffs = t1_names.difference(set(candidates_df['name'].unique()))
print(t1_t2_name_diffs)

{'Bob Dole', 'George S. McGovern'}


In [833]:
candidates_party_df.loc[(candidates_party_df['name']=='Bob Dole'), 'name'] = 'Robert Dole'

In [834]:
candidates_df.loc[(candidates_df['name_first']=='George')&(candidates_df['name_last']=='McGovern'), 'name'] = 'George S. McGovern'
candidates_df.loc[(candidates_df['name_first']=='George')&(candidates_df['name_last']=='McGovern'), 'name_middle'] = 'S.'

In [839]:
# Should be empty set now
t1_names = set(candidates_party_df['name'].unique())
t1_t2_name_diffs = t1_names.difference(set(candidates_df['name'].unique()))
print(f"Q: Are the candidate names derived from Table 2 and Table 1 spelled exactly the same? A: {len(t1_t2_name_diffs)==0}")

Q: Are the candidate names derived from Table 2 and Table 1 spelled exactly the same? A: True


#### **3.2.4 Construct Final Candidate Table**
Final Candidate table is created by left-joining the Candidates Party table to the Candidates State table

In [840]:
candidates_df = candidates_df.merge(candidates_party_df, how='left', on='name', copy=False, validate='1:1')

In [841]:
candidates_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 54
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         55 non-null     object
 1   name_first   55 non-null     object
 2   name_middle  35 non-null     object
 3   name_last    54 non-null     object
 4   name_suffix  2 non-null      object
 5   state        54 non-null     object
 6   state_2      2 non-null      object
 7   party        45 non-null     object
 8   party_2      2 non-null      object
dtypes: object(9)
memory usage: 4.3+ KB


In [842]:
candidates_df

,name,name_first,name_middle,name_last,name_suffix,state,state_2,party,party_2
0,Grover Cleveland,Grover,NaN,Cleveland,NaN,New York,None,D,None
1,Benjamin Harrison,Benjamin,NaN,Harrison,NaN,Indiana,None,R,None
2,James B. Weaver,James,B.,Weaver,NaN,Iowa,None,NaN,NaN
3,William McKinley,William,NaN,McKinley,NaN,Ohio,None,R,None
4,William J. Bryan,William,J.,Bryan,NaN,Nebraska,None,D,P
5,Theodore Roosevelt,Theodore,NaN,Roosevelt,NaN,New York,None,R,P
6,Alton B. Parker,Alton,B.,Parker,NaN,New York,None,D,None
7,William H. Taft,William,H.,Taft,NaN,Ohio,None,R,None
8,Woodrow Wilson,Woodrow,NaN,Wilson,NaN,New Jersey,None,D,None
9,Charles E. Hughes,Charles,E.,Hughes,NaN,New York,None,R,None


#### **3.2.5 Validate Final Candidate Table**
Make sure that no candidate, state, or party data has been lost along the way.

In [843]:
print(f"Q: Does each row correspond to a unique candidate name? A: {len(candidates_df) == len(candidates_df.drop_duplicates(subset='name'))}")

Q: Does each row correspond to a unique candidate name? A: True


In [844]:
num_final_candidates = len(candidates_df)
print(f"Q: Is the Final # of Candidates, {num_final_candidates}, equal to the # of Candidates from Table 2? A: {num_final_candidates==num_candidates}")

Q: Is the Final # of Candidates, 55, equal to the # of Candidates from Table 2? A: True


In [845]:
final_state_counts = candidates_df['state'].value_counts(dropna=False)
print(f"Q: Is the Final Distribution of Primary States equal to the Distribution of Primary States from Table 2? A: {(state_counts==final_state_counts).all()}")

Q: Is the Final Distribution of Primary States equal to the Distribution of Primary States from Table 2? A: True


In [846]:
final_state_2_counts = candidates_df['state_2'].value_counts(dropna=False)
print(f"Q: Is the Final Distribution of Secondary States equal to the Distribution of Secondary States from Table 2? A: {(state_2_counts==final_state_2_counts).all()}")

Q: Is the Final Distribution of Secondary States equal to the Distribution of Secondary States from Table 2? A: True


In [847]:
final_party_counts = candidates_df['party'].value_counts()
print(f"Q: Is the Final Distribution of Primary Party equal to the Distribution of Primary Party from Table 1? A: {(party_counts==final_party_counts).all()}")

Q: Is the Final Distribution of Primary Party equal to the Distribution of Primary Party from Table 1? A: True


In [848]:
final_party_2_counts = candidates_df['party_2'].value_counts()
print(f"Q: Is the Final Distribution of Secondary Party equal to the Distribution of Secondary Party from Table 1? A: {(party_2_counts==final_party_2_counts).all()}")

Q: Is the Final Distribution of Secondary Party equal to the Distribution of Secondary Party from Table 1? A: True


### 3.3 Transform and Validate State Data
Build state dataframe containing one row for each state, in alphabetical order, with the following information for each state:
- 'region' and 'division' corresponding to Census Bureau-designated regions and divisions, respectively
- 'statens' and 'geoid' corresponding to state ANSI code and FIPS code, respectively, which can be used to join to other datasets, e.g. Census population data, etc.
- 'name_usps' and 'name' corresponding to the state abbreviation and state name, respectively
- 'area_land' and 'area_water' corresponding to the land area and water area of the state, in m^2
- 'latitude' and 'longitude' corresponding to the latitude and longitude of the state's approximate centroid, in decimal degrees

#### **3.3.1 Extract, Validate ad Transform State Data**

#### Build State DataFrame

In [938]:
state_df = pd.DataFrame(sorted(us_state_names), columns=['name'])
state_df.info()

In [960]:
# state_df is derived from a set, so all name values must be unique
num_states = len(state_df)
print(f"Unique number of States = {num_states}")

Unique number of States = 51


In [940]:
state_df.head()

,name
0,Alabama
1,Alaska
2,Arizona
3,Arkansas
4,California


#### Join to GeoPandas DataFrame with State geographic info
Remove the following columns from the GeoPandas DataFrame prior to joining:
- 'STATEFP', as it is always the same as 'GEOID' column
- 'LSAD', as it is not relevant and always equal to 00
- 'MTFCC', as it is not relevant and always equal to G4000
- 'FUNCSTAT', as it is not relevant and always equal to A

In [941]:
usa.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   REGION    56 non-null     object  
 1   DIVISION  56 non-null     object  
 2   STATEFP   56 non-null     object  
 3   STATENS   56 non-null     object  
 4   GEOID     56 non-null     object  
 5   STUSPS    56 non-null     object  
 6   NAME      56 non-null     object  
 7   LSAD      56 non-null     object  
 8   MTFCC     56 non-null     object  
 9   FUNCSTAT  56 non-null     object  
 10  ALAND     56 non-null     int64   
 11  AWATER    56 non-null     int64   
 12  INTPTLAT  56 non-null     object  
 13  INTPTLON  56 non-null     object  
 14  geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(12)
memory usage: 6.7+ KB


In [942]:
usa.head()

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,62266231560,489271086,+38.6472854,-080.6183274,"POLYGON ((-81.74725 39.09538, -81.74635 39.096..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,138947364717,31362872853,+28.4574302,-082.4091477,"MULTIPOLYGON (((-86.38865 30.99418, -86.38385 ..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,143779863817,6215723896,+40.1028754,-089.1526108,"POLYGON ((-91.18529 40.63780, -91.17510 40.643..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,206230065476,18942261495,+46.3159573,-094.1996043,"POLYGON ((-96.78438 46.63050, -96.78434 46.630..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,25151726296,6979340970,+38.9466584,-076.6744939,"POLYGON ((-77.45881 39.22027, -77.45866 39.220..."


In [943]:
usa_cols_to_drop = ['STATEFP', 'LSAD', 'MTFCC', 'FUNCSTAT', 'geometry']
state_map_df = usa.drop(usa_cols_to_drop, axis=1, inplace=False)

In [944]:
state_map_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   REGION    56 non-null     object
 1   DIVISION  56 non-null     object
 2   STATENS   56 non-null     object
 3   GEOID     56 non-null     object
 4   STUSPS    56 non-null     object
 5   NAME      56 non-null     object
 6   ALAND     56 non-null     int64 
 7   AWATER    56 non-null     int64 
 8   INTPTLAT  56 non-null     object
 9   INTPTLON  56 non-null     object
dtypes: int64(2), object(8)
memory usage: 4.5+ KB


#### **3.3.2 Construct Final State Table**
Final State table is created using an inner join to the State GeoPandas DataFrame

In [945]:
state_df = state_df.merge(state_map_df, how='inner', left_on='name', right_on='NAME', validate='1:1')

In [946]:
state_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      51 non-null     object
 1   REGION    51 non-null     object
 2   DIVISION  51 non-null     object
 3   STATENS   51 non-null     object
 4   GEOID     51 non-null     object
 5   STUSPS    51 non-null     object
 6   NAME      51 non-null     object
 7   ALAND     51 non-null     int64 
 8   AWATER    51 non-null     int64 
 9   INTPTLAT  51 non-null     object
 10  INTPTLON  51 non-null     object
dtypes: int64(2), object(9)
memory usage: 4.8+ KB


#### Update Column Names

In [947]:
state_column_names = {'REGION': 'region', 'DIVISION': 'division', 'STATENS': 'statens', 'GEOID': 'geoid', 'STUSPS': 'name_usps', \
               'ALAND': 'area_land', 'AWATER': 'area_water', 'INTPTLAT': 'latitude', 'INTPTLON': 'longitude'}
state_df.drop(['NAME'], axis=1, inplace=True)
state_df.rename(state_column_names, axis=1, inplace=True)

#### Convert Appropriate Columns to Numeric Data Types
Convert Region, Division, Latitude and Longitude to numeric data types so they're formatted correctly for Postgres

In [955]:
state_df = state_df.astype({'region': 'int', 'division': 'int', 'latitude': 'float', 'longitude': 'float'}, copy=False)

#### **3.3.3 Validate Final State Table**
Make sure that no state data has been lost along the way.

In [956]:
state_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 50
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        51 non-null     object 
 1   region      51 non-null     int64  
 2   division    51 non-null     int64  
 3   statens     51 non-null     object 
 4   geoid       51 non-null     object 
 5   name_usps   51 non-null     object 
 6   area_land   51 non-null     int64  
 7   area_water  51 non-null     int64  
 8   latitude    51 non-null     float64
 9   longitude   51 non-null     float64
dtypes: float64(2), int64(4), object(4)
memory usage: 4.4+ KB


In [957]:
print(f"Q: Does each row correspond to a unique state name? A: {len(state_df) == len(state_df.drop_duplicates(subset='name'))}")

Q: Does each row correspond to a unique state name? A: True


In [963]:
num_final_states = len(state_df)
print(f"Q: Is the Final # of States, {num_final_states}, equal to the # of States? A: {num_final_states==num_states}")

Q: Is the Final # of States, 51, equal to the # of States? A: True


### 3.4 Transform and Validate Electoral Votes Data

#### **3.4.1 Extract, Validate ad Transform Electoral Votes Data**

#### Validate that each Election Year has the Correct # of States
Verify that the # of States that voted for President each year makes sense. I've confirmed that these values are consistent with when each state was added to the Union from 1892 to present, plus when Washington DC was allowed to vote (1964). See [this link](https://en.wikipedia.org/wiki/List_of_U.S._states_by_date_of_admission_to_the_Union) for details on when each state joined the Union and was able to cast electoral votes for President.

In [559]:
print("Year Index, Year Value, # of States Including Totals")
for ind, pyr in enumerate(parsed_election_years):
    print(ind, pyr['year'], len(pyr['t2']['votes_by_state']), sep=", ")

Year Index, Year Value, # of States Including Totals
0, 1892, 45
1, 1896, 46
2, 1900, 46
3, 1904, 46
4, 1908, 47
5, 1912, 49
6, 1916, 49
7, 1920, 49
8, 1924, 49
9, 1928, 49
10, 1932, 49
11, 1936, 49
12, 1940, 49
13, 1944, 49
14, 1948, 49
15, 1952, 49
16, 1956, 49
17, 1960, 51
18, 1964, 52
19, 1968, 52
20, 1972, 52
21, 1976, 52
22, 1980, 52
23, 1984, 52
24, 1988, 52
25, 1992, 52
26, 1996, 52
27, 2000, 52
28, 2004, 52
29, 2008, 52
30, 2012, 52
31, 2016, 52
32, 2020, 52


#### Build Initial Electoral Votes DataFrame

In [104]:
t2_votes_df = pd.json_normalize(parsed_election_years, ['t2', 'votes_by_state'], ['year'])

In [105]:
t2_votes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1649 entries, 0 to 1648
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   state                  1649 non-null   object 
 1   total_electoral_votes  1649 non-null   int64  
 2   1                      1649 non-null   int64  
 3   2                      1649 non-null   int64  
 4   3                      604 non-null    float64
 5   4                      52 non-null     float64
 6   year                   1649 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 90.3+ KB


In [106]:
t2_votes_df.head(10)

,state,total_electoral_votes,1,2,3,4,year
0,Alabama,11,11,0,0.0,NaN,1892
1,Arkansas,8,8,0,0.0,NaN,1892
2,California,9,8,1,0.0,NaN,1892
3,Colorado,4,0,0,4.0,NaN,1892
4,Connecticut,6,6,0,0.0,NaN,1892
5,Delaware,3,3,0,0.0,NaN,1892
6,Florida,4,4,0,0.0,NaN,1892
7,Georgia,13,13,0,0.0,NaN,1892
8,Idaho,3,0,0,3.0,NaN,1892
9,Illinois,24,24,0,0.0,NaN,1892


In [97]:
t2_votes_df = pd.melt(t2_votes_df, id_vars=['year', 'state', 'total_electoral_votes'], value_vars=[1, 2, 3, 4], \
              var_name='col_ind', value_name='president_electoral_votes')

In [98]:
t2_votes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6596 entries, 0 to 6595
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year                       6596 non-null   object 
 1   state                      6596 non-null   object 
 2   total_electoral_votes      6596 non-null   int64  
 3   col_ind                    6596 non-null   object 
 4   president_electoral_votes  3954 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 257.8+ KB


In [103]:
t2_votes_df.head(10)

,year,state,total_electoral_votes,col_ind,president_electoral_votes
0,1892,Alabama,11,1,11.0
1,1892,Arkansas,8,1,8.0
2,1892,California,9,1,8.0
3,1892,Colorado,4,1,0.0
4,1892,Connecticut,6,1,6.0
5,1892,Delaware,3,1,3.0
6,1892,Florida,4,1,4.0
7,1892,Georgia,13,1,13.0
8,1892,Idaho,3,1,0.0
9,1892,Illinois,24,1,24.0


## 4. Write Election Data to Postgres Database

In [111]:
# column_names = ['year', 'state', 'president_candidate_name', 'president_candidate_party', 'president_candidate_state', \
#                 'president_electoral_votes', 'president_electoral_rank', 'president_popular_votes', 'president_popular_rank']
# data_df = pd.DataFrame(columns=column_names)
# data_df.info()